# UNData API Exercise


In this exercise, you'll redo the data gathering phase of the UNData Exploration project by using APIs (Application Programming Interface) instead of downloading csv files.


In [1]:
import requests
import pandas as pd
import numpy as np
import json


You'll make use of the [World Bank Indicators API](https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-about-the-indicators-api-documentation). Note that this API does not require an API key. Before attempting the exercise, it would be a good idea to skim through the Documentation page and to check out the [Basic Call Structure article](https://datahelpdesk.worldbank.org/knowledgebase/articles/898581).


1. Use the API to get all available data for the _GDP per capita, PPP (constant 2021 international $)_ indicator. Hint: this indicator has code "NY.GDP.PCAP.PP.KD". Adjust the query parameters so that you can retrieve all available rows. Convert the results to a DataFrame.

In [8]:
endpoint = 'https://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.PP.KD'
params = {
    'format': 'json',
    'Per_page' : 296
}
response = requests.get(endpoint, params=params)
response.json()

[{'page': 1,
  'pages': 58,
  'per_page': 296,
  'total': 17024,
  'sourceid': '2',
  'lastupdated': '2024-10-24'},
 [{'indicator': {'id': 'NY.GDP.PCAP.PP.KD',
    'value': 'GDP per capita, PPP (constant 2021 international $)'},
   'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
   'countryiso3code': 'AFE',
   'date': '2023',
   'value': 4047.00703115979,
   'unit': '',
   'obs_status': '',
   'decimal': 0},
  {'indicator': {'id': 'NY.GDP.PCAP.PP.KD',
    'value': 'GDP per capita, PPP (constant 2021 international $)'},
   'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
   'countryiso3code': 'AFE',
   'date': '2022',
   'value': 4038.63868929989,
   'unit': '',
   'obs_status': '',
   'decimal': 0},
  {'indicator': {'id': 'NY.GDP.PCAP.PP.KD',
    'value': 'GDP per capita, PPP (constant 2021 international $)'},
   'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'},
   'countryiso3code': 'AFE',
   'date': '2021',
   'value': 3994.17165362078,

In [3]:
res = (
    pd
    .json_normalize(response.json()[1])
    .drop(
        columns={
            'countryiso3code',
            'unit',
            'obs_status',
            'decimal',
            'indicator.id',
            'indicator.value'})
    .rename(
        columns={
            'date':'year',
            'value':'gdp_per_capita',
            'country.id':'country_id',
            'country.value':'country_or_region'})
    .loc[:,['year', 'country_id', 'country_or_region', 'gdp_per_capita']]
)
print(res.shape)
res.head(5)


IndexError: list index out of range


2. Now, use the API to get all available data for _Life expectancy at birth, total (years)_. This indicator has code "SP.DYN.LE00.IN". Again, convert the results to a DataFrame.


In [4]:
endpoint = 'https://api.worldbank.org/v2/country/all/indicator/SP.DYN.LE00.IN'
params = {
    'format' : 'json',
    'Per_page': '341'
}
response2 = requests.get(endpoint, params=params)
response2

<Response [200]>

In [5]:
res2 = (
    pd
    .json_normalize(response2.json()[1])
    .drop(
        columns={
            'countryiso3code',
            'unit',
            'obs_status',
            'decimal',
            'indicator.id',
            'indicator.value'})
    .rename(
        columns={
            'date':'year',
            'value':'life_expectancy',
            'country.id':'country_id',
            'country.value':'country_or_region'})
    .loc[:,['year', 'country_id', 'country_or_region', 'life_expectancy']]
)
print(res2.shape)
res2.head(5)

(341, 4)


,year,country_id,country_or_region,life_expectancy
0,2023,ZH,Africa Eastern and Southern,NaN
1,2022,ZH,Africa Eastern and Southern,62.899031
2,2021,ZH,Africa Eastern and Southern,62.454590
3,2020,ZH,Africa Eastern and Southern,63.313860
4,2019,ZH,Africa Eastern and Southern,63.755678



3. Merge the two results DataFrames together. You may want to rename or drop columns prior to merging.


In [6]:
res_merged =(
    pd.merge(
        left=res,
        right=res2,
        on=['year', 'country_id', 'country_or_region'],
        how='outer')
)
print(res_merged.shape)
res_merged

(341, 5)


,year,country_id,country_or_region,gdp_per_capita,life_expectancy
0,1960,1A,Arab World,NaN,44.972899
1,1960,B8,Central Europe and the Baltics,NaN,67.933952
2,1960,S3,Caribbean small states,NaN,59.127246
3,1960,ZH,Africa Eastern and Southern,NaN,44.085552
4,1960,ZI,Africa Western and Central,NaN,37.845152
...,...,...,...,...,...
336,2023,B8,Central Europe and the Baltics,42391.250132,NaN
337,2023,S3,Caribbean small states,28099.087171,NaN
338,2023,V2,Early-demographic dividend,NaN,NaN
339,2023,ZH,Africa Eastern and Southern,4047.007031,NaN



4. You can also get more information about the available countries (region, capital city, income level classification, etc.) by using the [Country API](https://datahelpdesk.worldbank.org/knowledgebase/articles/898590-country-api-queries). Use this API to pull in all available data. Merge this with your other datasets. Use this to now remove the rows that correspond to regions and not countries.


In [7]:
endpoint='https://api.worldbank.org/v2/country'
params={
    'format':'json',
    'Per_page':'296'
}
response3 = requests.get(endpoint, params=params)
response3

<Response [200]>

In [10]:
res3 = pd.json_normalize(response3.json()[1]).rename(columns={'name':'country_or_region'})
res3
res_merged2 = pd.merge(left=res3, right=res_merged, on='country_or_region', how='left').replace(r'^\s*$', np.nan, regex=True).dropna(subset='capitalCity')
print(res_merged2.shape)
res_merged2.head()

(211, 22)


,id,iso2Code,country_or_region,capitalCity,longitude,latitude,region.id,region.iso2code,region.value,adminregion.id,...,incomeLevel.id,incomeLevel.iso2code,incomeLevel.value,lendingType.id,lendingType.iso2code,lendingType.value,year,country_id,gdp_per_capita,life_expectancy
0,ABW,AW,Aruba,Oranjestad,-70.0167,12.5167,LCN,ZJ,Latin America & Caribbean,NaN,...,HIC,XD,High income,LNX,XX,Not classified,NaN,NaN,NaN,NaN
65,AFG,AF,Afghanistan,Kabul,69.1761,34.5228,SAS,8S,South Asia,SAS,...,LIC,XM,Low income,IDX,XI,IDA,NaN,NaN,NaN,NaN
131,AGO,AO,Angola,Luanda,13.242,-8.81155,SSF,ZG,Sub-Saharan Africa,SSA,...,LMC,XN,Lower middle income,IBD,XF,IBRD,NaN,NaN,NaN,NaN
132,ALB,AL,Albania,Tirane,19.8172,41.3317,ECS,Z7,Europe & Central Asia,ECA,...,UMC,XT,Upper middle income,IBD,XF,IBRD,NaN,NaN,NaN,NaN
133,AND,AD,Andorra,Andorra la Vella,1.5218,42.5075,ECS,Z7,Europe & Central Asia,NaN,...,HIC,XD,High income,LNX,XX,Not classified,NaN,NaN,NaN,NaN


## **Bonus Questions:** 

After doing a basic request to get all records, you can attempt these exercises to get additional practice using query parameters.



1. Adjust your request so that it returns data just for the United States.



2. Adjust your request so that it returns data just for the United States for the year 2021.



3. Adjust your request so that it returns data just for the United States for the years 2000 through 2021.



4. Adjust your request so that it returns data for the United States and Canada for the years 2000 through 2021.



5. If you haven't already done so and you would like to get some additional practice using loops, use the page parameter in order to pull all records. Do not change the value of the per_page parameter. You will likely need to utilize a loop of some kind in order to pull all records.



6. The endpoint "<http://api.worldbank.org/v2/indicator>" will return information about all available indicators. See [Indicator API Queries](https://datahelpdesk.worldbank.org/knowledgebase/articles/898599-indicator-api-queries). Use this endpoint to find the code for the "Public Expenditure on Education (% GDP)" indicator.